In [ ]:
!pip install optuna category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 19.7 MB/s eta 0:00:00


In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import optuna

# 데이터 로드 (파일 경로 변경)
df = pd.read_csv('/content/drive/MyDrive/chicago dataset/final_crime_data.csv')

# 결측치 처리
df.dropna(inplace=True)
df.drop(['Case Number', 'Date', 'Block', 'IUCR', 'Description',
       'FBI Code', 'Updated On', 'Location'], axis=1, inplace=True)

# 타겟 변수(Primary Type) 분리 및 인코딩
y = df['Primary Type']
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

X = df.drop('Primary Type', axis=1)

# 범주형 컬럼 처리 준비
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Location Description 빈도 인코딩
high_card_col = 'Location Description'
cat_cols.remove(high_card_col)

freq = X[high_card_col].value_counts(normalize=True)
X[high_card_col] = X[high_card_col].map(freq)

# 나머지 범주형 변수는 레이블 인코딩
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42)

# Optuna 목적함수 정의
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
    }

    gbm = GradientBoostingClassifier(**params, random_state=42)
    gbm.fit(X_train, y_train)
    preds = gbm.predict(X_test)
    accuracy = accuracy_score(y_test, preds)

    return accuracy

# Optuna로 최적화
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# 결과 출력
print("최적 하이퍼파라미터:", study.best_params)
print("최고 정확도:", study.best_value)

# 최적 모델로 재학습 및 평가
best_gbm = GradientBoostingClassifier(**study.best_params, random_state=42)
best_gbm.fit(X_train, y_train)
final_preds = best_gbm.predict(X_test)

print("최종 정확도:", accuracy_score(y_test, final_preds))

# 예측값 원본 라벨로 변환 및 확인
final_preds_labels = le_target.inverse_transform(final_preds)
print("예측값 샘플:", final_preds_labels[:10])


[I 2025-04-07 05:37:50,683] A new study created in memory with name: no-name-e3232724-d6b2-4eb8-8047-b2e36fd9fea7
[W 2025-04-07 08:54:37,919] Trial 0 failed with parameters: {'n_estimators': 51, 'learning_rate': 0.03971736979857507, 'max_depth': 8, 'subsample': 0.8217977509595153, 'min_samples_split': 13} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-5-2e838d3280c5>", line 55, in objective
    gbm.fit(X_train, y_train)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_gb.py", line 787, in fit
    n_stages = self._fit_stages(
               ^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [ ]:

# 1. 라이브러리 불러오기
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier


# 2. 데이터 불러오기
df = pd.read_csv("/content/drive/MyDrive/chicago dataset/final_crime_data.csv")

# 3. 기본 전처리
data = df.copy()

# 날짜 처리
data["Date"] = pd.to_datetime(data["Date"], errors="coerce")
data["Month"] = data["Date"].dt.month
data["DayOfWeek"] = data["Date"].dt.dayofweek
data["Hour"] = data["Date"].dt.hour

# 불필요한 열 제거
drop_cols = [
    "Unnamed: 0", "ID", "Case Number", "Date", "Block", "IUCR", "Description",
    "FBI Code", "Updated On", "Location"
]
data = data.drop(columns=drop_cols)


# 4. 희귀 클래스(100건 이하) 제거
type_counts = data["Primary Type"].value_counts()
valid_types = type_counts[type_counts > 100].index
data = data[data["Primary Type"].isin(valid_types)].reset_index(drop=True)


# 5. 인코딩 처리
# 타겟 변수 인코딩
label_encoder = LabelEncoder()
data["Primary Type"] = label_encoder.fit_transform(data["Primary Type"])
y = data["Primary Type"]

# Arrest, Domestic → bool → int
data["Arrest"] = data["Arrest"].astype(bool).astype(int)
data["Domestic"] = data["Domestic"].astype(bool).astype(int)

# Label Encoding
for col in ["Community Area", "District", "Ward", "Beat"]:
    data[col] = label_encoder.fit_transform(data[col])

# Frequency Encoding
location_counts = data["Location Description"].value_counts()
data["Location Description"] = data["Location Description"].map(location_counts)

# 누락값 처리
data = data.fillna(-999)

# X 데이터 분리
X = data.drop(columns=["Primary Type"])


# 6. 학습/검증 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


# 7. Optuna 튜닝 함수 정의
def objective(trial):
    params = {
        "objective": "multi:softmax",
        "num_class": len(np.unique(y)),
        "eval_metric": "mlogloss",
        "use_label_encoder": False,
        "tree_method": "hist",
        "n_jobs": -1,
        "random_state": 42,
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
    }

    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    return acc


# 8. Optuna 최적화 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("Best params:", study.best_params)
print("Best score:", study.best_value)


# 9. 최적 파라미터로 모델 학습
best_params = study.best_params
best_params.update({
    "objective": "multi:softmax",
    "num_class": len(np.unique(y)),
    "eval_metric": "mlogloss",
    "use_label_encoder": False,
    "tree_method": "hist",
    "n_jobs": -1,
    "random_state": 42
})

final_model = XGBClassifier(**best_params)
final_model.fit(X_train, y_train)


# 10. 최종 평가
y_pred = final_model.predict(X_valid)
acc = accuracy_score(y_valid, y_pred)
print("최종 정확도:", acc)


[I 2025-04-08 08:53:13,670] A new study created in memory with name: no-name-001d4e2a-055d-4bc9-af16-f07f3c1e6f4c
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:53:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
